# Scraper

In [1]:
import sys
import time
import undetected_chromedriver.v2 as uc
import copy
import logging
import json
import warnings
warnings.filterwarnings("ignore")
import requests

In [2]:
def enterLeolist(driver):
    with driver:
        driver.get("https://www.leolist.cc/personals/female-escorts/new-brunswick")
    print("Current URL: ", driver.current_url)
    # Enter to leolist.
    close = input("Do you have chrome://welcome/ open? Y/N ")
    if close == "Y":
        chrome_window = driver.window_handles[1]
        driver.switch_to.window(chrome_window)
        driver.close()
        leolist_window = driver.window_handles[0]
        driver.switch_to.window(leolist_window)
        time.sleep(2)
    agree_button = driver.find_element_by_xpath("/html/body/div[4]/div/div/div/div[2]/a")
    agree_button.click()

In [3]:
# Crear driver.
print("Open Chrome")
driver = uc.Chrome()
with driver:
    driver.get("https://www.leolist.cc/personals/female-escorts/greater-toronto/city_toronto_downtown_fun_thick_sweet_and_juicy_cum_see_me_now-6998683?source=list")

Open Chrome


In [5]:
dates = driver.find_elements_by_class_name("ers_tooltip")

In [10]:
dates[0].get_attribute("data-original-title")

'Posted on 2022-04-07 04:11:18'

In [8]:
dates[1].text

''

In [9]:
dates[2].text

''

In [58]:
API_KEY = "0968ef0f2a24ff3d1b8ba332afd980f4"
GEETEST_ID = "72bf15796d0b69c43867452fea615052"

In [59]:
def solveGeetestV4(pageurl:str) -> dict:
    route = "http://2captcha.com/in.php?key={api_key}&method=geetest_v4&captcha_id={captcha_id}&&pageurl={pageurl}" \
            .format(api_key = API_KEY, captcha_id = GEETEST_ID, pageurl = pageurl)
    #logging.warning("Post request: ", route)
    print("Post request: ", route)
    result = requests.post(route)
    req_id = result.text[3:]
    route = "http://2captcha.com/res.php?key={api_key}&action=get&id={req_id}" \
            .format(api_key = API_KEY, req_id = req_id)
    
    # Wait 20 seconds for 2recaptcha response.
    time.sleep(20)
    #logging.warning("Get request: ", route)
    print("Get request: ", route)
    response = requests.get(route)
    tries = 0
    
    # Recaptcha is not ready, repeat request every 5 seconds until MAX_TRIES
    while response.text == "CAPCHA_NOT_READY":
        time.sleep(5)
        while tries > MAX_TRIES:
            logging.error("Recaptcha max tries exceeded.")
            sys.exit()
        tries = tries + 1
        response = requests.get(route)

    string = response.text[3:]
    data = json.loads(string)
    return data

In [48]:
pageurl = "https://www.leolist.cc/personals/female-escorts/greater-toronto/city_toronto_downtown_fun_thick_sweet_and_juicy_cum_see_me_now-6998683?source=list"
data = solveGeetestV4(pageurl)

Post request:  http://2captcha.com/in.php?key=0968ef0f2a24ff3d1b8ba332afd980f4&method=geetest_v4&captcha_id=72bf15796d0b69c43867452fea615052&&pageurl=https://www.leolist.cc/personals/female-escorts/greater-toronto/city_toronto_downtown_fun_thick_sweet_and_juicy_cum_see_me_now-6998683?source=list


ConnectionError: HTTPConnectionPool(host='2captcha.com', port=80): Max retries exceeded with url: /in.php?key=0968ef0f2a24ff3d1b8ba332afd980f4&method=geetest_v4&captcha_id=72bf15796d0b69c43867452fea615052&&pageurl=https://www.leolist.cc/personals/female-escorts/greater-toronto/city_toronto_downtown_fun_thick_sweet_and_juicy_cum_see_me_now-6998683?source=list (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EF93E462B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
data

In [ ]:
def inputHidden(driver, field, value):
    script = "return document.getElementById('{field}').value = '{value}'".format(field = field, value = value)
    print(script)
    driver.execute_script(script)
#inputHidden(driver, "gee_test_captcha_output", "random")

In [24]:
#script = "return document.getElementById({field}).input[0].value = '{value}'".format(field = field, value = value)
#print(script)
#result = driver.execute_script(script)

In [ ]:
registerForm = driver.find_element_by_id("registerForm")

In [ ]:
control_groups = registerForm.find_elements_by_class_name("control-group")

In [ ]:
elements = control_groups[7].find_elements_by_tag_name("input")
elements

In [ ]:
elements[0].get_attribute("name")

In [ ]:
#gee_test_captcha_output
#gee_test_gen_time
#gee_test_pass_token
#gee_test_lot_number

In [49]:
data

{'captcha_id': '72bf15796d0b69c43867452fea615052',
 'lot_number': 'a04f851833a54cc99ac51949b7804df1',
 'pass_token': '91dd4455b75e68e6fab5e8b1ae7ed3f98f2dc43756e8c6a995b8531ff2fba35b',
 'gen_time': '1650101625',
 'captcha_output': 'QMSMLCB4Fs58XZrBrecNrWUsiqdZPixqLOEmX7qpQM8ArwpscIasPUC-t8ZsVzto8_GDz1FD4kP-mE1KShaGaEA7dEufUAn1RH8HVGPEC5l-TlYKN50_P6BUQ8dUoZxGO0LRCW3CnTGpFp-0jUsoiIJiWnKwYDjJibhpQ9GDWnCxFt-lhZtmmjgEZ8_oxnQD'}

In [25]:
import hmac

In [40]:
def post(data):

    # 1, Initialize GeeTest parameter information 
    captcha_id = data["captcha_id"]
    captcha_key = API_KEY
    api_server = 'http://gcaptcha4.geetest.com'

    # 2, Get the verification parameters passed from the front end after user verification  
    lot_number = data['lot_number']
    captcha_output = data['captcha_output']
    pass_token = data['pass_token']
    gen_time = data['gen_time']

    # 3. Generate signature
    # Using standard hmac algorithms to generate signatures, using the user's current verification serial number lot_number as the original message, and the client's verification private key as the key 
    # Using sha256 hash algorithm to hash message and key in one direction to generate the final signature 
    lotnumber_bytes = lot_number.encode()
    prikey_bytes = captcha_key.encode()
    sign_token = hmac.new(prikey_bytes, lotnumber_bytes, digestmod='SHA256').hexdigest()

    # 4. Upload verification parameters to the second verification interface of GeeTest to verify the user verification status 
    query = {
        "lot_number": lot_number,
        "captcha_output": captcha_output,
        "pass_token": pass_token,
        "gen_time": gen_time, 
        "sign_token": sign_token
    } 

    # captcha_idParameter is recommended to be placed after url, so that when an exception is requested, it can be quickly located in the log according to the id 
    url = api_server + '/validate' + '?captcha_id={}'.format(captcha_id)
    
    print(url)
    print(query)

    # Pay attention to handling interface exceptions, and make corresponding exception handling when requesting GeeTest secondary verification interface exceptions or response status is not 200 
    # Guarantee that the business process will not be blocked by interface request timeout or service non-response 
    try:
        res = requests.post(url, query)
        assert res.status_code == 200
        gt_msg = json.loads(res.text)
        print(gt_msg)
    except Exception as e:
        gt_msg = {'result': 'success', 'reason': 'request geetest api fail'}

    # 5. According to the user authentication status returned by GeeTest, the website owner conducts his own business logic 
    return gt_msg

In [41]:
post(data)

http://gcaptcha4.geetest.com/validate?captcha_id=72bf15796d0b69c43867452fea615052
{'lot_number': 'a04f851833a54cc99ac51949b7804df1', 'captcha_output': 'QMSMLCB4Fs58XZrBrecNrWUsiqdZPixqLOEmX7qpQM8ArwpscIasPUC-t8ZsVzto8_GDz1FD4kP-mE1KShaGaEA7dEufUAn1RH8HVGPEC5l-TlYKN50_P6BUQ8dUoZxGO0LRCW3CnTGpFp-0jUsoiIJiWnKwYDjJibhpQ9GDWnCxFt-lhZtmmjgEZ8_oxnQD', 'pass_token': '91dd4455b75e68e6fab5e8b1ae7ed3f98f2dc43756e8c6a995b8531ff2fba35b', 'gen_time': '1650101625', 'sign_token': '10842bb3afbb50b0462ea0dcc307ffbb95e8447add0f42e5f1a7bf1f887d93c3'}
{'result': 'fail', 'reason': 'sign_token error', 'captcha_args': {'used_type': 'slide', 'user_ip': '113.170.97.4', 'lot_number': 'a04f851833a54cc99ac51949b7804df1', 'scene': 'Others', 'referer': 'https://www.leolist.cc/'}}


{'result': 'fail',
 'reason': 'sign_token error',
 'captcha_args': {'used_type': 'slide',
  'user_ip': '113.170.97.4',
  'lot_number': 'a04f851833a54cc99ac51949b7804df1',
  'scene': 'Others',
  'referer': 'https://www.leolist.cc/'}}

In [50]:
def submitGeetestV4(driver: uc.Chrome, data: dict) -> None:
    inputHidden(driver, "gee_test_lot_number", data["lot_number"])
    inputHidden(driver, "gee_test_pass_token", data["pass_token"])
    inputHidden(driver, "gee_test_gen_time", data["gen_time"])
    inputHidden(driver, "gee_test_captcha_output", data["captcha_output"])
    #driver.execute_script("return document.getElementById('registersubmit').click()")

In [51]:
submitGeetestV4(driver, data)

return document.getElementById('gee_test_lot_number').value = 'a04f851833a54cc99ac51949b7804df1'
return document.getElementById('gee_test_pass_token').value = '91dd4455b75e68e6fab5e8b1ae7ed3f98f2dc43756e8c6a995b8531ff2fba35b'
return document.getElementById('gee_test_gen_time').value = '1650101625'
return document.getElementById('gee_test_captcha_output').value = 'QMSMLCB4Fs58XZrBrecNrWUsiqdZPixqLOEmX7qpQM8ArwpscIasPUC-t8ZsVzto8_GDz1FD4kP-mE1KShaGaEA7dEufUAn1RH8HVGPEC5l-TlYKN50_P6BUQ8dUoZxGO0LRCW3CnTGpFp-0jUsoiIJiWnKwYDjJibhpQ9GDWnCxFt-lhZtmmjgEZ8_oxnQD'


In [52]:
def clickPhoneButton(driver):
    button = None
    list_a = driver.find_elements_by_class_name("contacts-view-btn")
    for b in list_a:
        if b.text.startswith("click to view") or b.text.startswith("SHOW"):
            try:
                button = copy.copy(b)
                button.click()
                break
            except: 
                pass

In [53]:
clickPhoneButton(driver)

In [54]:
driver.execute_script("return document.getElementById('registersubmit').click()")